In [1]:
import pandas
import sqlalchemy

import server.model.connection as sm_connection
import server.model.event as event
import server.model.match as sm_match
import server.view.dataframes as sv_dataframes
import server.view.excel as sv_excel

In [2]:
event.EventDal.set_current_event("waamv", "2018")
event.EventDal.set_current_match("070-q")

'current match 070-q'

In [ ]:
def measure_summary_df(num_matches=12):
    # Connect to database
    conn = sm_connection.engine.connect()

    # Get current event
    evt_id = event.EventDal.get_current_event()[0]

    select_sum = sqlalchemy.text(
        "WITH current AS "
        "(SELECT status.event_id as event_id, status.match, schedules.date "
        "FROM status INNER JOIN schedules "
        "ON status.event_id=schedules.event_id AND "
        "status.match=schedules.match "
        "WHERE date <> 'na' LIMIT 1), "

        "recent_matches as "
        "(SELECT * FROM ( "
        "SELECT row_number() "
        "over (partition by team order by sched.date desc) as r, "
        " sched.* from schedules sched, current c "
        "WHERE sched.event_id = c.event_id and sched.date <= c.date )"
        " row_schedule "
        "WHERE row_schedule.r <= " + str(
            num_matches) + " ORDER by team, date desc), "

       "team_match_count as ( "
       "SELECT team, count(team) AS team_matches FROM recent_matches "
       "GROUP BY team) "

       "SELECT teams.name AS team, phases.name AS phase, tasks.name AS task,"
       "actors.name AS actor, "
       "MAX(team_match_count.team_matches) AS matches, "
        
       "SUM(successes) AS sum_successes, MAX(successes) as max_successes, "
       "MIN(successes) AS min_successes, COUNT(successes) AS count_successes, "
       "CAST(SUM(successes) AS FLOAT)/MAX(team_match_count.team_matches) AS avg_successes, "
       "AVG(successes) AS tav_successes, "
        
       "SUM(attempts) AS sum_attempts, MAX(attempts) as max_attempts, "
       "MIN(attempts) AS min_attempts, COUNT(attempts) AS count_attempts, "
       "CAST(SUM(attempts) AS FLOAT)/MAX(team_match_count.team_matches) as avg_attempts, "
       "AVG(attempts) AS tav_attempts, "
        
       "SUM(cycle_times) AS sum_cycle_times,"
       "MAX(cycle_times) AS max_cycle_times, "
       "MIN(cycle_times) AS min_cycle_times, "
       "CAST(SUM(cycle_times) AS FLOAT)/MAX(team_match_count.team_matches) AS avg_cycle_times, "
       "AVG(cycle_times) AS tav_cycle_times, "
       "COUNT(cycle_times) AS count_cycle_times, "
        
       "SUM(capability) AS sum_capabilities, "
       "MAX(capability) as max_capabilities, "
       "MIN(capability) AS min_capabilities, "
       "COUNT(capability) AS count_capabilities, "
       "CAST(SUM(capability) AS FLOAT)/MAX(team_match_count.team_matches) as avg_capabilities, "
       "AVG(capability) AS tav_capabilities "
        
       "FROM (((((((teams FULL OUTER JOIN measures "
       "ON teams.id=measures.team_id) "
       "LEFT JOIN tasks ON tasks.id = measures.task_id) "
       "LEFT JOIN phases ON phases.id = measures.phase_id) "
       "LEFT JOIN events ON events.id = measures.event_id) "
       "LEFT JOIN actors ON actors.id = measures.actor_id) "
       "LEFT JOIN matches ON matches.id = measures.match_id) "
       "LEFT JOIN team_match_count ON team_match_count.team = teams.name) "
       "RIGHT JOIN recent_matches ON recent_matches.match = matches.name AND "
       "team_match_count.team = teams.name "
       "AND recent_matches.team = team_match_count.team "
       "WHERE events.id = :evt_id "
       "GROUP BY teams.name, tasks.name, phases.name, actors.name "
       "ORDER BY teams.name, phases.name, tasks.name, actors.name;"
    ).bindparams(evt_id=evt_id)
    summ_df = pandas.read_sql(select_sum, conn)
    summ_df.set_index(['team', 'phase', 'actor', 'task'], inplace=True)
    conn.close()
    return summ_df

In [3]:
df = sv_dataframes.measure_summary_df(12)

In [4]:
df.head()

matches  sum_successes  max_successes  \
team phase  actor task                                                   
1258 auto   robot autoLine            12              9              1   
                  crossNull           12              0              0   
                  holdCube            12              8              1   
                  startPosition       12              0              0   
     finish robot disabled            12              1              1   

                                 min_successes  count_successes  \
team phase  actor task                                            
1258 auto   robot autoLine                   1                9   
                  crossNull                  0                1   
                  holdCube                   1                8   
                  startPosition              0                7   
     finish robot disabled                   1                1   

                                 avg_successes  tav_successes  sum_attempts  \
team phase  actor task                                                        
1258 auto   robot autoLine            0.750000            1.0             9   
                  crossNull           0.000000            0.0             0   
                  holdCube            0.666667            1.0             8   
                  startPosition       0.000000            0.0             0   
     finish robot disabled            0.083333            1.0             1   

                                 max_attempts  min_attempts        ...         \
team phase  actor task                                             ...          
1258 auto   robot autoLine                  1             1        ...          
                  crossNull                 0             0        ...          
                  holdCube                  1             1        ...          
                  startPosition             0             0        ...          
     finish robot disabled                  1             1        ...          

                                 min_cycle_times  avg_cycle_times  \
team phase  actor task                                              
1258 auto   robot autoLine                     0              0.0   
                  crossNull                    0              0.0   
                  holdCube                     0              0.0   
                  startPosition                0              0.0   
     finish robot disabled                     0              0.0   

                                 tav_cycle_times  count_cycle_times  \
team phase  actor task                                                
1258 auto   robot autoLine                   0.0                  9   
                  crossNull                  0.0                  1   
                  holdCube                   0.0                  8   
                  startPosition              0.0                  7   
     finish robot disabled                   0.0                  1   

                                 sum_capabilities  max_capabilities  \
team phase  actor task                                                
1258 auto   robot autoLine                      0                 0   
                  crossNull                     0                 0   
                  holdCube                      0                 0   
                  startPosition              8773              1254   
     finish robot disabled                      0                 0   

                                 min_capabilities  count_capabilities  \
team phase  actor task                                                  
1258 auto   robot autoLine                      0                   9   
                  crossNull                     0                   1   
                  holdCube                      0                   8   
                  startPosition              1253                   7   
    

In [ ]:
def ranking_df(num_matches):
    summ_df = measure_summary_df(num_matches)
    rank_df = summ_df.stack()
    rank_stacked_df = rank_df.unstack([1, 2, 3, 4])
    return rank_stacked_df.sort_index(axis= 1, level= [0,1,2])

In [5]:
dfr = sv_dataframes.ranking_df(12)

In [6]:
dfr["teleop"]["robot"]["placeSwitch"]

,matches,sum_successes,max_successes,min_successes,count_successes,avg_successes,tav_successes,sum_attempts,max_attempts,min_attempts,...,min_cycle_times,avg_cycle_times,tav_cycle_times,count_cycle_times,sum_capabilities,max_capabilities,min_capabilities,count_capabilities,avg_capabilities,tav_capabilities
team,,,,,,,,,,,,,,,,,,,,,
1258,12.0,4.0,2.0,0.0,4.0,0.333333,1.000000,4.0,2.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0
1318,12.0,5.0,5.0,0.0,3.0,0.416667,1.666667,5.0,5.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0
1425,12.0,4.0,3.0,0.0,4.0,0.333333,1.000000,5.0,3.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0
1778,12.0,10.0,7.0,0.0,4.0,0.833333,2.500000,12.0,8.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0
1899,12.0,3.0,1.0,0.0,7.0,0.250000,0.428571,7.0,2.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0
2046,12.0,5.0,2.0,0.0,4.0,0.416667,1.250000,9.0,4.0,1.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,0.0
2557,12.0,8.0,2.0,0.0,7.0,0.666667,1.142857,10.0,3.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0
2906,12.0,4.0,2.0,0.0,5.0,0.333333,0.800000,6.0,2.0,1.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,0.0
2907,12.0,6.0,1.0,0.0,7.0,0.500000,0.857143,12.0,3.0,1.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0


In [ ]:
27*26